In [1]:
from keras import layers,Input
from keras.models import Model

Using TensorFlow backend.


In [3]:
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])


In [5]:
model.compile(optimizer='rmsprop',
              loss=['mse',
                    'categorical_crossentropy',
                    'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])


In [6]:
# model.compile(optimizer='rmsprop',
#               loss={'age': 'mse',
#                     'income': 'categorical_crossentropy',
#                     'gender': 'binary_crossentropy'},
#               loss_weights={'age': 0.25,
#                             'income': 1.,
#                             'gender': 10.})


In [26]:
import numpy as np
import keras

num_samples = 1000
max_length = 100
posts = np.random.randint(1, vocabulary_size, size=(num_samples, max_length))
age_targets = np.random.randint(10, 100, size=num_samples)
income_targets = np.random.randint(num_income_groups, size=num_samples)
income_targets = keras.utils.to_categorical(income_targets, num_income_groups)
gender_targets = np.random.randint(0, 1, size=num_samples)

model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10,
          batch_size=64,
          validation_split=.2)


Train on 800 samples, validate on 200 samples
Epoch 1/10


InvalidArgumentError: Computed output size would be negative: -2 [input_size: 2, effective_filter_size: 5, stride: 1]
	 [[{{node conv1d_6/convolution/Conv2D}} = Conv2D[T=DT_FLOAT, _class=["loc:@train...kpropInput"], data_format="NHWC", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv1d_6/convolution/ExpandDims, conv1d_6/convolution/ExpandDims_1)]]